## Mapping drought in WA
##### update these details for your code
Author:  Stanley Mastrantonis

Date: 

version 1 python 3, written in ESRI ArcPro v2.9.

## Mapping drought in WA
  
We will use this data to map the BoM definition of drought across WA for a set time period.  The BoM definition of drought refers to a year of total rainfall below the 10th percentile for the history of data. 

What is a data cube and arrays: 

![data cube](https://docs.xarray.dev/en/stable/_static/dataset-diagram-logo.png)

Data cubes are multi-dimensional extensions of two-dimensional tables.

A data cube can be thought of as a set of similar 2D tables stacked on top of each other. For example, a set of 2D tables showing population at 30 June by age group for each local government area would form a data cube, with local government area as the third dimension.

What is gdal:

![gdal](https://gdal.org/_images/OSGeo_project.png)

GDAL is a C/C++/Python translator library for more than 200 raster and vector geospatial data formats. 

gdal instances or bindings are used by almost all spatial software, and provide the functions for projecting data between CRSs. 

## Libraries and modules 

In [1]:
import numpy as np
import urllib, urllib.request
import xarray
from osgeo import gdal, gdalconst, osr
import os, copy, math, glob
import arcpy #ESRI
from arcgis.gis import GIS #ESRI 
gis = GIS('pro')
from matplotlib import pyplot as plt

## Functions 
read in the functions from the py script or the below cell

In [3]:
#from Drought_functions import *

In [4]:
def maskarray(file: str, xmin: float, xmax: float, ymin: float, ymax: float) -> np.ndarray:
    """
    Masks an xarray dataset based on longitude and latitude ranges and returns the sum of values along the 'time' dimension.
    
    Args:
        file (str): File path to the xarray dataset.
        xmin (float): Minimum longitude value for masking.
        xmax (float): Maximum longitude value for masking.
        ymin (float): Minimum latitude value for masking.
        ymax (float): Maximum latitude value for masking.
    
    Returns:
        np.ndarray: Numpy array containing the sum of values after masking along the 'time' dimension.
    """
    xds_i = xr.open_dataset(file)
    mask_lon_i = (xds_i.lon >= xmin) & (xds_i.lon <= xmax)
    mask_lat_i = (xds_i.lat >= ymin) & (xds_i.lat <= ymax)
    clipped_i = xds_i.where(mask_lon_i & mask_lat_i, drop=True)
    xds_i_sum = clipped_i.sum(skipna=True, dim='time').drop_vars(names='crs', errors='ignore')
    
    nparr = xds_i_sum.to_array().values                     
    return nparr

def arrToRast(np_arr: np.ndarray, res: float, display: bool = True) -> None:
    """
    Creates a raster from a numpy array using arcpy.

    Args:
        np_arr (np.ndarray): Numpy array to be converted to a raster.
        res (float): Resolution of the raster.
        display (bool, optional): Flag to add the output raster to the map display. Defaults to True.

    Returns:
        None
    """
    wgs = arcpy.SpatialReference('WGS 1984')  # create an arcpy spatial reference
    arcpy.env.addOutputsToMap = display
    rast_arr = arcpy.NumPyArrayToRaster(np_arr, ll, res, res, 0)  # convert numpy array to arcpy raster

    arcpy.management.DefineProjection(rast_arr, wgs)  # set the coordinate reference system (CRS) of the new raster

    if 'rain' in str(os.getcwd()).lower():
        nm = '_Rain.tif'
    else:
        nm = '_Drought.tif'
    rast_arr.save(os.path.join(os.getcwd(), mt_i + nm))  # save the raster
    del rast_arr

def flipyflip(nparr: np.ndarray) -> np.ndarray:
    """
    Flips a numpy array twice.

    Args:
        nparr (np.ndarray): Numpy array to be flipped.

    Returns:
        np.ndarray: Flipped numpy array.
    """
    np_xarr_fl = np.flip(np.flip(nparr, 2), 2)
    return np_xarr_fl

## Folder structure
The following code will create a folder structure in a working folder for wherever the 'cwd' variable is set to 

In [ ]:
cwd = '' #Change this to your working directory
datdir = os.path.join(cwd, 'Data')
datfolds = os.path.join(datdir, 'Creation')

if os.path.exists(datdir): 
        pass 
else: 
        os.mkdir(datdir)

if os.path.exists(datfolds): 
        pass 
else: 
        os.mkdir(datfolds)

folders = ['netCDF','rasters','shapefiles','Arcpyenv', 'WA' ]

for folder in folders:
    if os.path.exists(os.path.join(datfolds, folder)): 
        pass 
    else: 
        os.mkdir(os.path.join(datfolds, folder))
        
rastout = os.path.join(datfolds, 'rasters')        
rast_folds = ['rain','stack','drought','severe drought','scratch']

for folder in rast_folds:
    if os.path.exists(os.path.join(rastout, folder)): 
        pass 
    else: 
        os.mkdir(os.path.join(rastout, folder))

In [ ]:
#arcpy environment changes
arcpy.env.workspace = os.path.join(datfolds, 'Arcpyenv')
arcpy.env.overwriteOutput = True

## Read in the WA shapefile and set create a WGS84 crs 

Make sure you move the WA shapefile to the correct folder ("WA") in your newly created Data directory

In [ ]:
roi = os.path.join(datfolds, 'WA\\WA_gcs.shp')
sr = arcpy.SpatialReference("WGS 1984")
arcpy.management.DefineProjection(roi, sr)

## Extracting extents of WA

In [ ]:
extent = arcpy.Describe(roi).extent
ll = extent.lowerLeft # we need the origin point of the bounding layer (WA) to build rasters later on
print (extent) 
xmin = extent.XMin
xmax = extent.XMax
ymin = extent.YMin
ymax = extent.YMax

## What is an AWS server bucket?
For us, it's a url. Run the below code to inspect the SILO AWS bucket and its content:

In [6]:
%%html
<iframe src='https://s3-ap-southeast-2.amazonaws.com/silo-open-data/Official/annual/index.html' width="1200" height="1000"></iframe>

## Specify a year range
We need to specify a time period to collect data from the AWS bucket.
For now lets do 10 years of data. You can try for the whole period of data, but just remember it still takes time to download and convert every file

In [ ]:
year_range = list(range(2010,2021,1))
print(year_range)

## Download and convert the SILO rainfall data
The following code will query and request the specified nc files. nc files are NetCDF files which are data cubes. 
The nc files have data for all of Australia and we will mask this data for WA.
The code will convert the nc files into an xarray dataset. We will then mask the xarray array with the function maskarray called above. How is it masking the xarray array?
The code will also convert the xarray array to to a numpy array and then to a raster with the arrToRast function.
The arrToRast function was called above and relies on the arpcy NumPyArrayToRaster function. We will learn how to convert an array to a raster with gdal later.

Note: if for some reason the SILO data will not download, the nc files have been provided for you and you can skip to the next cell to convert the nc files to rasters

Please note that it can take some time to download all the data

In [ ]:
for i in range(0, len(year_range), 1):
    mt_i = str(year_range[i])
    rain_i = 'https://s3-ap-southeast-2.amazonaws.com/silo-open-data/Official/annual/monthly_rain/'+mt_i+'.monthly_rain.nc' 
    #we are querying every year of data between our year range
    print(rain_i)
    rain_i_file = os.path.join(datdir, 'Creation//netCDF') + '//Rain_'+mt_i+'.nc'
    if os.path.isfile(rain_i_file): 
        os.remove(rain_i_file)
        urllib.request.urlretrieve(rain_i, rain_i_file)
    else:
        urllib.request.urlretrieve(rain_i, rain_i_file)    

The below cell will mask the nc files to WA and convert them to raster files.
The raster files will be saved in the rain folder under the rasters directory. Temporary raster files will also be added to the map. You can disable this in the arrToRast function by changing the display argument to False. 

You will notice that the rasters are the wrong way around. This often happens with array data and you will need to find a solution:

Hint: search for numpy.flip in the API documentation
The answers have also been provided in the answers Notebook. 

Be sure to remove the temporary rasters from your contents pane.

In [ ]:
ncfiles = glob.glob(os.path.join(datdir, 'Creation//netCDF//*.nc')) 
os.chdir(os.path.join(rastout, 'rain'))

for i in range(0, len(year_range), 1):
    mt_i = str(year_range[i])
    nc_file = ncfiles[i]
    marray = maskarray(nc_file)#maskthe array
    arrToRast(marray, 0.05, display = True) # create a raster from the array


del marray
os.chdir(cwd)  

## Sense check you results
Below is a link to BoM drought average maps. Do your results look similar?
Again, your raster may be the wrong way around
Hint: lookup the numpy.flip() function

In [5]:
%%html
<iframe src='http://www.bom.gov.au/climate/averages/climatology/rainfall/hires_rn/aus/rnozan.png' width="1000" height="800"></iframe>

## Calculating percentiles and exporting a raster file with gdal
Now that we have several years of rainfall data in raster (.tif) format, we can calculate the 10th percentile of all the data using gdal and numpy.
We then use gdal to save the percentile array as a raster file

In [ ]:
filenames = glob.glob(os.path.join(rastout, 'rain\\*.tif')) # This will list all files of a certain type in a folder
array_list = [] #create an empty list to store our arrays


gsrc = gdal.Open(filenames[1]) #open a single raster to extract its attributes
geotransform = gsrc.GetGeoTransform() 
projection = gsrc.GetProjectionRef()
    
for file in filenames:
    src = gdal.Open(file)
    array_list.append(src.ReadAsArray()) #we will append the raster values as numpy arrays to the array_list

    
stacked_array = np.stack(array_list) #Here, we stack all the arrays into a multidientional numpy array 
decile_arr = np.percentile(stacked_array, q = 10, axis = 0) # We calculate the 10th percentile for the raster stack


driver = gdal.GetDriverByName('GTiff')
rows, cols = decile_arr.shape
perc = driver.Create(os.path.join(rastout,'stack\\Percentile.tif'),
                        cols, rows, 1,
                        gdal.GDT_Float32)

perc.SetGeoTransform(geotransform)
perc.SetProjection(projection)
perc.WriteArray(decile_arr)
perc.GetRasterBand(1).SetNoDataValue(0)

stats = perc.GetRasterBand(1).GetStatistics(0,1)
perc.FlushCache()


plt.figure()
plt.imshow(decile_arr, interpolation = 'nearest')
plt.show()

del perc, stacked_array, array_list, src, gsrc

lets look at some summary statistics for the 10th percentile raster

In [ ]:
stats_dict = {'Max': str(stats[1]), 'Mean': str(stats[2]),
              'Min': str(stats[0]), 'STDEV' : str(stats[3])}
print(stats_dict)

Here, we are classifying every year of the rainfall data as drought if it falls below the 10th percentile threshold for any point in space.

The classified rasters (1 for drought) will be saved in the drought folder in the 
rasters directory

If you do not want the temporary rasters to be displayed change the display argument to false in the arrToRast function

Note that because we are classifying drought on the same period we are calculating the percentile, all of WA will be in drought at least once out of the 10 year period. Ideally, you would calculate the percentile for the entire period of data and classify a defined period (BoM suggests 40 years).

You can of course try for a longer time frame, but for now we will stick to 10 years for the purposes of the lesson. 

In [ ]:
os.chdir(os.path.join(rastout, 'drought'))
drought_arr = []

psrc = gdal.Open(os.path.join(rastout+'\\stack\\Percentile.tif'))\
            .ReadAsArray()

for i, file in enumerate(filenames):
    mt_i = str(year_range[i])
    rainarr = gdal.Open(file).ReadAsArray()
    drought = np.where(np.subtract(rainarr,psrc) < 0,1,0)
    drought_arr.append(drought)
    arrToRast(drought, 0.05, display = True)
    
os.chdir(cwd) 
del rainarr, drought